# EDA216 - Lecture 2

During the lectures and exercises we'll try out SQL-code in Jupyter
Notebooks, and to do that we begin by evaluating:

In [1]:
%load_ext sql

This time I'll use a database defined for a Stanford course:

In [2]:
%sql sqlite:///db02.db

'Connected: None@db02.db'

Everything in the code sections is evaluated
as [Python](https://www.python.org/) code (more on that later), but
lines beginning with `%sql` will be seen as an SQL statement, and all
lines following `%%sql` will also be evaluated as SQL statements.

## Joins

The following example is taken from Jennifer Widom, with some name
alterations (to comply with the style I've been using).

We have the following schema:

~~~~ {.sql}
CREATE TABLE colleges(c_name text, state text, enrollment int);
CREATE TABLE students(s_id int, s_name text, gpa real, size_hs int);
CREATE TABLE applications(s_id int, c_name text, major text, decision text);
~~~~


A typical selection:

In [3]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s, applications a
WHERE  s.s_id = a.s_id

Done.


s_name,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


The comma in the `FROM` clause gives us a cross join:

In [4]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s CROSS JOIN applications a
WHERE  s.s_id = a.s_id

Done.


s_name,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


This can be rewritten as a *inner join* on the `s_id`:

In [5]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s INNER JOIN applications a
ON     s.s_id = a.s_id

Done.


s_name,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


and as `INNER JOIN` is the default join operator, we can omit the
`INNER`:

In [6]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s JOIN applications a
ON     s.s_id = a.s_id

Done.


s_name,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


We can add conditions to our inner joins, we start with another
typical cross join:

In [7]:
%%sql
SELECT s_name, gpa
FROM   students s, applications a
WHERE  s.s_id = a.s_id AND
       size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Done.


s_name,gpa
Helen,3.7
Irene,3.9


This can be rewritten to a `INNER JOIN` on the id's, and the rest of
the condition can be a `WHERE` clause:

In [8]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a
ON     s.s_id = a.s_id
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Done.


s_name,gpa
Helen,3.7
Irene,3.9


but we can also put all conditions in the `ON` clause:

In [9]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a
ON     s.s_id = a.s_id AND
       size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Done.


s_name,gpa
Helen,3.7
Irene,3.9


It will make little difference, the query engine is supposed to
optimize our queries either way.

Now let's try with three tables -- we want to see some application
information:

In [11]:
%%sql
SELECT a.s_id, s_name, gpa, a.c_name, enrollment
FROM   applications a, students s, colleges c
WHERE  a.s_id = s.s_id AND
       a.c_name = c.c_name

Done.


s_id,s_name,gpa,c_name,enrollment
123,Amy,3.9,Stanford,15000
123,Amy,3.9,Stanford,15000
123,Amy,3.9,Berkeley,36000
123,Amy,3.9,Cornell,21000
234,Bob,3.6,Berkeley,36000
345,Craig,3.5,MIT,10000
345,Craig,3.5,Cornell,21000
345,Craig,3.5,Cornell,21000
345,Craig,3.5,Cornell,21000
678,Fay,3.8,Stanford,15000


We can again rewrite this as joins:

In [12]:
%%sql
SELECT a.s_id, s_name, gpa, a.c_name, enrollment
FROM   applications a JOIN students s JOIN colleges c
ON     a.s_id = s.s_id AND
       a.c_name = c.c_name

Done.


s_id,s_name,gpa,c_name,enrollment
123,Amy,3.9,Stanford,15000
123,Amy,3.9,Stanford,15000
123,Amy,3.9,Berkeley,36000
123,Amy,3.9,Cornell,21000
234,Bob,3.6,Berkeley,36000
345,Craig,3.5,MIT,10000
345,Craig,3.5,Cornell,21000
345,Craig,3.5,Cornell,21000
345,Craig,3.5,Cornell,21000
678,Fay,3.8,Stanford,15000


## Other kinds of joins

Back to our original query:

In [13]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s INNER JOIN applications a
ON     s.s_id = a.s_id

Done.


s_name,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


Since our two tables have the attribute `s_id` in common, we can use a
*natural join*:

In [14]:
%%sql
SELECT DISTINCT s_name, major
FROM   students NATURAL JOIN applications

Done.


s_name,major
Amy,CS
Amy,EE
Bob,biology
Craig,CS
Craig,EE
Craig,bioengineering
Fay,history
Helen,CS
Irene,CS
Irene,biology


The natural join will eliminate the duplicate attributes:

In [15]:
%%sql
SELECT *
FROM   students NATURAL JOIN applications

Done.


s_id,s_name,gpa,size_hs,c_name,major,decision
123,Amy,3.9,1000,Berkeley,CS,Y
123,Amy,3.9,1000,Cornell,EE,Y
123,Amy,3.9,1000,Stanford,CS,Y
123,Amy,3.9,1000,Stanford,EE,N
234,Bob,3.6,1500,Berkeley,biology,N
345,Craig,3.5,500,Cornell,CS,Y
345,Craig,3.5,500,Cornell,EE,N
345,Craig,3.5,500,Cornell,bioengineering,N
345,Craig,3.5,500,MIT,bioengineering,Y
678,Fay,3.8,200,Stanford,history,Y


Observe that there's only one `s_id` in the output.

We can use natural joins with more complicated queries:

In [16]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a
ON     s.s_id = a.s_id
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Done.


s_name,gpa
Helen,3.7
Irene,3.9


Once again we have a common attribute, so we can write:

In [17]:
%%sql
SELECT s_name, gpa
FROM   students s NATURAL JOIN applications a
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Done.


s_name,gpa
Helen,3.7
Irene,3.9


Using natural join is a bit risky, since we might have other
attributes with the same name (it can easily happen over time, if we
modify our tables) -- a better way of expressing this is to use a
`JOIN` with `USING`:

In [18]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a USING(s_id)
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Done.


s_name,gpa
Helen,3.7
Irene,3.9


Here we explicitly tell what attributes we want to equate.

A slightly bigger example -- we want to find all pairs with the same
Grade Point Average (GPA):

In [19]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students s1, students s2
WHERE  s1.gpa = s2.gpa AND
       s1.s_id < s2.s_id

Done.


s_id,s_name,gpa,s_id_1,s_name_1,gpa_1
123,Amy,3.9,456,Doris,3.9
123,Amy,3.9,654,Amy,3.9
123,Amy,3.9,876,Irene,3.9
456,Doris,3.9,654,Amy,3.9
456,Doris,3.9,876,Irene,3.9
567,Edward,2.9,765,Jay,2.9
654,Amy,3.9,876,Irene,3.9
543,Craig,3.4,789,Gary,3.4


In [22]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students s1 JOIN students s2 USING(gpa)
WHERE  s1.s_id < s2.s_id

Done.


s_id,s_name,gpa,s_id_1,s_name_1,gpa_1
123,Amy,3.9,456,Doris,3.9
123,Amy,3.9,654,Amy,3.9
123,Amy,3.9,876,Irene,3.9
456,Doris,3.9,654,Amy,3.9
456,Doris,3.9,876,Irene,3.9
567,Edward,2.9,765,Jay,2.9
654,Amy,3.9,876,Irene,3.9
543,Craig,3.4,789,Gary,3.4


We will return to joins when we get into relational algebra later in
the course.